In [14]:
from pynq import Overlay

ol = Overlay('./dsp_pynq.bit')

list(ol.ip_dict.keys())

['DAS_data',
 'DSA_X',
 'DSA_Xe',
 'DSA_Y',
 'DAScompound64_0',
 'processing_system7_0']

In [18]:
ol.DAS_data.register_map
import numpy
import csv
x = numpy.loadtxt(open("x.csv","rb"),dtype=numpy.int16,delimiter=',',skiprows=0)
y = numpy.loadtxt(open("y.csv","rb"),dtype=numpy.int16,delimiter=',',skiprows=0)
xele = numpy.loadtxt(open("xele.csv","rb"),dtype=numpy.int16,delimiter=',',skiprows=0)
Din = numpy.loadtxt(open("DIn.csv","rb"),dtype=numpy.int16,delimiter=',',skiprows=0)

DAS_data = ol.DAS_data
DSA_X = ol.DSA_X
DSA_Xe = ol.DSA_Xe
DSA_Y = ol.DSA_Y
print(len(x))
print(len(y))
print(len(xele))
print(len(Din[0]))
print(len(Din))

253
390
64
64
2560


In [34]:
from pynq import allocate
# Create DMA buffer for coefs
#fir_buffer_reload = allocate(shape=(len(hpf_coeffs_hw),),dtype=np.int16)

# Copy coefs to buffer
#for i in range(len(hpf_coeffs_hw)):
 #   fir_buffer_reload[i] = hpf_coeffs_hw[i]

# Transfer coefficients to FIR
#dma_reload.sendchannel.transfer(fir_buffer_reload)
#dma_reload.sendchannel.wait()

# Send an empty 8-bit packet to FIR config port to complete reload
#control_buffer_config = allocate(shape=(1,),dtype=numpy.int32)
#control_buffer_config[0] = 0
#control.sendchannel.transfer(control_buffer_config)
#control.sendchannel.wait()

# Close the buffers
#control_buffer_config.close()

# Create DMA buffer
x_in_buffer = allocate(shape=(len(x),),dtype=numpy.int16)
y_in_buffer = allocate(shape=(len(y),),dtype=numpy.int16)
xele_in_buffer = allocate(shape=(1,),dtype=numpy.int16)
din_buffer = allocate(shape=(len(Din),),dtype=numpy.int16)
data_out_buffer = allocate(shape=(len(x)*len(y),),dtype=numpy.int16)

# Copy data to buffer
for i in range(len(x)):
    x_in_buffer[i] = x[i]
for j in range(len(y)):
    y_in_buffer[j] = y[j]
for k in range(len(Din)):
    din_buffer[k] = Din[k,0]
#xele_in_buffer = xele[0]

# Transfer
#for i in range(0,max_iters):
DSA_X.sendchannel.transfer(x_in_buffer)
DSA_Y.sendchannel.transfer(y_in_buffer)
DSA_Xe.sendchannel.transfer(xele[0])
AS_data.sendchannel.transfer(din_buffer)
DAS_data.recvchannel.transfer(data_out_buffer)

DSA_X.sendchannel.wait()
DSA_Y.sendchannel.wait()
DSA_Xe.sendchannel.wait()
DAS_data.sendchannel.wait()
DAS_data.recvchannel.wait()

data = data_out_buffer
numpy.savetxt('Hilbert.csv',data,delimiter=',')

RuntimeError: DMA channel not idle